In [17]:
import math
import random
import fastText # updated fastText for one step SGD
import time

In [11]:
# Algorithm for model selection framework
#
# @input T = total amount of requests
# @input c = constant hyperparameter c > 1
# p_t = probability of querying all models/exploration
# X_t = binary value, explore or not explore
# t_c = content for current request
def algorithm(T, c):
    for t in range(1, T+1):
        p_t = min(1, c/math.sqrt(t))
    
        X_t = Bernoulli(p_t)
        if X_t == 1:
            # queryAll()
            sgdStep(t_c) 
        else:
            queryBest()

In [12]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [13]:
# def queryAll():
#     large_predictor = fasttext.load_model("large_predictor.bin")
#     small_predictor = fasttext.load_model("small_predictor.bin")

In [14]:
def queryBest():
    pass

In [15]:
def sgdStep():
    large_predictor = fasttext.load_model("large_predictor.bin")
    small_predictor = fasttext.load_model("small_predictor.bin")

    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{entry['large_model_accuracy']} {entry['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{entry['small_model_accuracy']} {entry['input_text']}\n")

    model = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    model = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    model.save_model("large_predictor.bin")
    model.save_model("small_predictor.bin")

In [16]:
algorithm(50, 2)

NameError: name 't_c' is not defined